### 분류의 성능 평가 지표
- 정확도(Accuracy)
- 오차행렬(Confusion Matrix)
- 정밀도(Precision)
- 재현율(Recall)
- F1 스코어
- ROC AUC

#### 위에 언급된 분류의 성능지표는 이진/멀티 분류 모두 적용되지만 특히 이진분류에서 더욱 중요하게 강조하는 지표이다.

### 정확도
- 정확도는 직관적으로 모델 예측 성능을 나타내는 평가지표.
- 데이터의 구성에 따라 ML모델의 성능을 왜곡할 수 있기 때문에 정확도 수치 하나만 가지고 성능을 평가하지는 않는다.
- ML알고리즘을 적용한 후 예측 정확도가 80%대 였지만 탑승객이 남자인 경우보다 여자인 경우에 생존확률이 높았기 때문에 무조건 성별이 여자인 경우 생존으로, 남자인 경우 사망으로 예측 결과를 예측해도 이와 비슷한 수치가 나올 수 잇다.
- 사이킷런의 BaseEstimator 클래스를 상속받아 아무런 학습 하지 않고 성별에 따라 생존자를 예측하는 단순한 Classifier를 생성한다.
- 사이킷런은 BaseEstimator를 상속받으면 Customized 형태의 Estimator를 개발자가 생성할 수 있다.
- 단순히 Sex 피처가 1이면 0, 그렇지않으면 1로 예측하는 매우 단순한 Classifier.

In [11]:
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
    # fit() 메소드는 아무것도 학습하지 않는다.
    def fit(self, X, y=None):
        pass
    # predict() 메소드는 단순히 Sex 피처가 1이면 0, 그렇지 않으면 1로 예측
    def predict(self, X):
        pred = np.zeros( (X.shape[0], 1))
        for i in range(X.shape[0]):
            if X['Sex'].iloc[i] == 1:
                pred[i] = 0
            else:
                pred[i] = 1
        return pred
    

- 생성된 MyDummyClassifier를 이용하여 타이타닉 생존자 예측을 수행해본다.


In [12]:
# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

# 불필요한 속성삭제
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'], axis=1, inplace=True)
    return df

# 레이블 인코딩
def format_features(df):
    from sklearn.preprocessing import LabelEncoder
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df



In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

titanic_df = pd.read_csv("../ch02/train.csv")
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived',axis=1)
X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=42)



In [14]:
import numpy as np
myclf = MyDummyClassifier()
myclf.fit(X_train, y_train)

mypredictions = myclf.predict(X_test)
print('Dummy Classifier의 정확도는 : {:.4f}'.format(accuracy_score(y_test, mypredictions)))

Dummy Classifier의 정확도는 : 0.7821


- 이렇게 단순한 알고리즘으로 예측해도 78%로 꽤 높은 수치가 나온다.
- 그러므로 정확도를 평가지표로 사용할 때는 신중해야한다.
- 특히 imbalanced 레이블 값 분포에서 ML모델의 성능을 판단할 때 적합한 평가지표가 아니다
- 예를들어 100개의 데이터에 90개의 레이블이 0, 10개만 1이라고 하면 무조건 0으로 예측결과를 반환하는  ML 모델의 경우 정확도가 90%가 된다.
- 예제코드로 확인해 본다.


In [18]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator):
    def fit(self, X, y):
        pass
    
    # 입력값으로 들어오는걸 모두 0으로 만들어 반환
    def predict(self, X):
        return np.zeros( (len(X), 1), dtype=bool)
    
digits = load_digits()

y = (digits.target == 7).astype(int)
X_train, X_test, y_train, y_test = train_test_split( digits.data, y, random_state=42)

In [19]:
print('레이블 테스트 세트 크기 :', y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도')
print(pd.Series(y_test).value_counts())

fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fakepred = fakeclf.predict(X_test)
print('모든 예측을 0으로 하여도 정확도는 : {:.3f}'.format(accuracy_score(y_test, fakepred)))

레이블 테스트 세트 크기 : (450,)
테스트 세트 레이블 0과 1의 분포도
0    409
1     41
dtype: int64
모든 예측을 0으로 하여도 정확도는 : 0.909


- 단순히 모두 0 값으로 반환함에도 불구하고 450개의 테스트 데이터 세트에 수행한 예측 정확도는 90%를 넘는다.
- 이처럼 정확도 평가 지표는 불균형한 레이블 데이터 세트에서는 성능 수치로 사용돼서는 안된다.
- 이런 한계점을 극복하기 위해 여러가지 분류지표와 함께 적용한다.
- True/False, Positive/Negative의 오차행렬이 있다.


### 오차행렬  
TN FP  
FN FP  

- 행이 실제 클래스 열이 예측 클래스


- TN : 예측값을 Negative값 0 으로 예측했고 실제 값 역시 Negative 값 0
- FP : 예측값을 Positive값 1 로 예측했는데 실제 값은 Negative 값 0
- FN : 예측값을 Negative값 0 으로 예측했는데 실제 값은 Positive 값 1
- TP : 예측값을 Positive값 1 로 예측했는데 실제 값 역시 Positive 값 1  





#### 위의 MyFakeClassifier의 예측 성능 지표를 오차 행렬로 표현해 본다.


In [20]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, fakepred)

array([[409,   0],
       [ 41,   0]])

- TN은 409
- FP는 0
- FN은 41
- TP는 0
